# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

***Participantes (RM - NOME):***<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>
xxxx - xxxxx<br>

### **Crie um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para alguma área especialista no assunto para uma melhor tratativa.​

1. Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do seu modelo.​

  O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

2. Utilizar ao menos uma aplicação de modelos de GenAI (LLM´s) para criar o modelo classificador com os mesmos critérios do item 1.

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, aplicações de GenIA, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais serão descontado nota e passível de reprovação.

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

In [ ]:
# CARREGANDO O DATA FRAME
import pandas as pd
#df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df = pd.read_csv('tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [ ]:
df.info()

Bom desenvolvimento!

### **Area de desenvolvimento e validações**

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

### Parte 01 - Uso de ML para Classificação de Textos

#### Exploração & Pré-processamento dos Textos

Além de explorar os dados, aqui aplicamos técnicas clássicas de normalização e limpeza de texto, como:

- Lowercasing
- Remoção de pontuação, números e padrões via regex (ex: "xx/xx/xxxx", "xxxx", valores monetários)
- Remoção de acentos
- Tokenização
- Remoção de stopwords (incluindo personalização)
- Filtro de palavras curtas

O objetivo é preparar os textos para vetorização e modelagem.

In [ ]:
df.isnull().sum()

In [ ]:
df['categoria'].value_counts()
df['categoria'].value_counts(normalize=True) * 100  # Em %


In [ ]:
# verificanr tamanho dos textos 

df['tamanho_texto'] = df['descricao_reclamacao'].apply(lambda x: len(str(x).split()))
df['tamanho_texto'].describe()
df_alt = df


In [ ]:
# explorando alguns textos por categoria
for categoria in df['categoria'].unique():
    exemplo = df[df['categoria'] == categoria]['descricao_reclamacao'].iloc[0]
    print(f"\n >>> Categoria: {categoria}\nTexto: {exemplo}")


#### Pré-processamento

In [ ]:
!pip install nltk unicode

In [ ]:
import nltk
nltk.download('punkt_tab') # obtive problemas com o punkt -> essa versao punkt_tab é obsoleta pela documentação
nltk.download('stopwords')

In [ ]:
import string
import nltk
import re
from nltk.tokenize import word_tokenize

# Lista de stopwords em português
stopwords = nltk.corpus.stopwords.words('portuguese')

# Função para remover pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# Normalização e tokenização
def norm_tokenize(text):
    text = str(text).lower()
    
    # Limpeza com regex 
    text = re.sub(r'xx+/xx+/xxxx?', ' ', text)  # datas como xx/xx/xxxx
    text = re.sub(r'\$?\s?\d+(?:,\d+)?', ' ', text)  # valores monetários
    text = re.sub(r'(?i)\b[x]{2,}\b', ' ', text)  # palavras com xxxx, xx etc
    text = re.sub(r'\d+', ' ', text)  # remove números restantes
    text = re.sub(r'\s+', ' ', text)  # espaços duplicados

    # Remover pontuação
    text = remove_punctuation(text)

    # Tokenização
    text = word_tokenize(text)

    # Remoção de stopwords e palavras curtas
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]

    return " ".join(text)

# Aplicando o pré-processamento
df_alt['texto_processado'] = df_alt['descricao_reclamacao'].apply(norm_tokenize)

# Mostrando antes e depois para alguns exemplos
df_alt[['descricao_reclamacao', 'texto_processado']].head()


##### Teste de Vetorizadores e Modelos de Classificação

Nesta etapa, comparamos diferentes combinações de vetorizadores e algoritmos de classificação para identificar o pipeline com melhor desempenho.

- **TF-IDF (1-2):** Considera unigramas e bigramas com normalização TF-IDF
- **TF-IDF (1-3):** Considera unigramas, bigramas e trigramas (mais contexto)
- **BoW (1-2):** Modelo tradicional de contagem com unigramas e bigramas

Modelos de classificação testados:
- **Naive Bayes (MultinomialNB):** 
- **Logistic Regression:** 
- **LinearSVC:** 
- **Random Forest:** 

Parâmetros:
- Todos os vetores foram limitados a **5.000 features** (usando `max_features=5000`)
- Dados separados em **75% treino / 25% teste** com `random_state=42` e `stratify=y`
- Métrica usada: **F1 Score (weighted)** — ideal para bases com classes desbalanceadas

A seguir, apresentamos uma tabela com os resultados ordenados pelo maior F1 Score.




In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd

X = df_alt['texto_processado']
y = df_alt['categoria']

vetorizadores = {
    'TF-IDF (1-2)': TfidfVectorizer(ngram_range=(1, 2), max_features=5000),
    'TF-IDF (1-3)': TfidfVectorizer(ngram_range=(1, 3), max_features=5000),
    'BoW (1-2)': CountVectorizer(ngram_range=(1, 2), max_features=5000),
}

modelos = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'LinearSVC': LinearSVC(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

resultados = []

for nome_vet, vet in vetorizadores.items():
    X_vet = vet.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_vet, y, test_size=0.25, random_state=42, stratify=y
    )

    for nome_modelo, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted')

        resultados.append({
            'Vetorizador': nome_vet,
            'Modelo': nome_modelo,
            'F1 Score': round(f1, 4)
        })

# Exibir resultados como DataFrame
df_resultados = pd.DataFrame(resultados)
print(df_resultados.sort_values(by='F1 Score', ascending=False))


In [ ]:
# Visualizar tabela de resutlados
df_resultados = pd.DataFrame(resultados).sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

# Estilizar a visualização
styled = df_resultados.style.set_caption("Comparativo de Vetorizadores e Modelos") \
    .background_gradient(subset=['F1 Score'], cmap='Greens') \
    .format({'F1 Score': '{:.4f}'}) \
    .hide(axis='index') \
    .set_table_styles([{
        'selector': 'caption',
        'props': [('color', '#333'), ('font-size', '16px'), ('font-weight', 'bold')]
    }])

styled


###### Nosso modelo campeão escolhido foi Regressão Logísica com 0.90 de F1 utilizando o TF-IDF(1-2) como vetorizador

### Parte 2 — Classificação de Texto com GenAI (Embeddings com Transformers)

Nesta etapa, vamos aplicar técnicas de PLN com modelos baseados em Transformers para classificar os chamados dos clientes da QuantumFinance.

Utilizaremos embeddings gerados com o modelo `paraphrase-multilingual-MiniLM-L12-v2`, que representa frases inteiras como vetores semânticos densos.

O pipeline será:

1. Carregamento e separação dos dados
2. Geração de embeddings com modelo Transformer
3. Treinamento de modelo de ML (ex: Logistic Regression)
4. Avaliação com F1 Score
5. Comparação com o modelo da Parte 1


In [3]:
# Carregamento dos dados e separação em treino/teste

import pandas as pd
from sklearn.model_selection import train_test_split

# Carrega o dataset
df = pd.read_csv('tickets_reclamacoes_classificados.csv', sep=';')

# Visualiza as primeiras linhas
print("Amostra do dataset:")
display(df.head())

# Verifica colunas e tipos
print("\nInformações do dataset:")
print(df.info())

# Remove linhas com valores nulos
df = df.dropna(subset=['descricao_reclamacao', 'categoria'])

# Separação das features (texto) e labels (categorias)
X = df['descricao_reclamacao']
y = df['categoria']

# Split em treino e teste (75/25), como pedido
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)


Amostra do dataset:


,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."



Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB
None


##### Geração de Embeddings com modelo Transformer

Usaremos o modelo `paraphrase-multilingual-MiniLM-L12-v2` da biblioteca `sentence-transformers`.

Esse modelo é baseado em BERT e é treinado para gerar **representações semânticas de sentenças** == embeddings. Eificiente para classificacoes e etc


In [4]:
#!pip install -U sentence-transformers -q

In [5]:
import torch
from sentence_transformers import SentenceTransformer

# Verifica se a GPU está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Dispositivo em uso: {device}")

# Carrega o modelo diretamente na GPU (se disponível)
#model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)
model = SentenceTransformer('neuralmind/bert-base-portuguese-cased', device=device)


# Gera os embeddings para os textos de treino e teste
X_train_embeddings = model.encode(X_train.tolist(), show_progress_bar=True)
X_test_embeddings = model.encode(X_test.tolist(), show_progress_bar=True)


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


Dispositivo em uso: cuda


Batches:   0%|          | 0/494 [00:00<?, ?it/s]

Batches:   0%|          | 0/165 [00:00<?, ?it/s]

###### Avaliação de Múltiplos Modelos com Embeddings

- Agora que já temos os embeddings prontos, vamos testar diversos modelos de machine learning para verificar qual deles apresenta o melhor desempenho na tarefa de classificação.



In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

# Lista de modelos para testar
modelos = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    #'LinearSVC': LinearSVC(),
    'RandomForest': RandomForestClassifier()
    #'KNN': KNeighborsClassifier(),
    #'GaussianNB': GaussianNB()  # pode funcionar mal com vetores densos, mas vamos ver
}

# Avaliação
resultados = []

for nome, modelo in modelos.items():
    try:
        modelo.fit(X_train_embeddings, y_train)
        y_pred = modelo.predict(X_test_embeddings)
        f1 = f1_score(y_test, y_pred, average='weighted')
        resultados.append((nome, f1))
        print(f'{nome:<20} → F1 Score: {f1:.2%}')
    except Exception as e:
        print(f'{nome:<20} → Erro: {e}')




LogisticRegression   → F1 Score: 80.69%
RandomForest         → F1 Score: 64.93%


#### Parte 2.2 - Fine tunning completo do BERT 

Os resultados acima são satisfatórios, mas ainda podemos explorarr algumas coisas:
- em vez de usar apenas os embeddings, vamos treinar o modelo BERT completo, incluindo sua camada de saída, diretamente para a tarefa de classificação.
- Ajustar os pesos internos de um modelo LLM já pré-treinado, usando um conjunto de dados específico (nossos chamados), para que ele aprenda a resolver melhor a tarefa desejada (classificação de categorias).


In [7]:
# Parte 2.2 — Fine-Tuning com BERT (LLM completo)

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import nltk
import re, string

# Download de recursos do NLTK
nltk.download('stopwords')
stopwords_list = nltk.corpus.stopwords.words('portuguese')

# Lista para substituição de nome de instituições
nomes = ['chase', 'bank', 'jp', 'gm', 'financial', 'jpmcb']
novo_nome = '[INST]'

# Normalização leve, sem remover stopwords que são essenciais para LLMs
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def normalize_text(text, replace_institutions=False):
    text = re.sub(r'\d+|/', '', text)
    text = re.sub(r'\bx\b|\w*xx+\w*', '', text)  # remove 'xxx', 'xxxx'
    text = remove_punctuation(text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    if replace_institutions:
        for inst in nomes:
            text = re.sub(r'\b' + re.escape(inst) + r'\b', novo_nome, text) 
    
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# >> Tokenizando com BERT 
# Carrega o tokenizer oficial do BERT em português (cased)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Adiciona o token especial usado para nomes de instituições
if "[INST]" not in tokenizer.get_vocab():
    tokenizer.add_tokens(["[INST]"])

# Função de tokenização completa
def bert_tokenize(text):
    text = normalize_text(text, replace_institutions=True)
    return tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )


In [9]:
# Preparação dos dados para fine-tuning:
# Codificação dos rótulos, tokenização em lote e criação dos tensores

from tqdm import tqdm

# X e y já carregados anteriormente
# X = df['descricao_reclamacao']
# y = df['categoria'])

# Codifica os rótulos (categorias) com números
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Aplica normalização e tokenização em todos os textos
input_ids_list = []
attention_masks_list = []

print("Tokenizando os textos com BERT...")

for texto in tqdm(X):
    tokens = bert_tokenize(texto)
    input_ids_list.append(tokens['input_ids'].squeeze(0))
    attention_masks_list.append(tokens['attention_mask'].squeeze(0))

# Converte listas em tensores
input_ids_tensor = torch.stack(input_ids_list)
attention_mask_tensor = torch.stack(attention_masks_list)
labels_tensor = torch.tensor(y_encoded, dtype=torch.long)


Tokenizando os textos com BERT...


100%|██████████| 21072/21072 [00:52<00:00, 404.74it/s]


In [10]:
#Separação em treino e teste + DataLoaders

# Divide os dados em treino/teste com base nos tensores
train_idx, test_idx = train_test_split(
    range(len(X)),
    test_size=0.25,
    random_state=42,
    stratify=y_encoded
)

# Cria datasets com base nos índices
train_dataset = TensorDataset(
    input_ids_tensor[train_idx],
    attention_mask_tensor[train_idx],
    labels_tensor[train_idx]
)

test_dataset = TensorDataset(
    input_ids_tensor[test_idx],
    attention_mask_tensor[test_idx],
    labels_tensor[test_idx]
)

# DataLoaders para treinamento e avaliação
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Agora temos:
#train_loader e test_loader com os dados prontos

# Tokenização feita com BERT
# Labels codificados e vetores de entrada prontos

In [11]:
input_ids_tensor

tensor([[  101,  8399,   644,  ...,     0,     0,     0],
        [  101, 13103,   535,  ...,     0,     0,     0],
        [  101,   231, 12807,  ...,     0,     0,     0],
        ...,
        [  101,  2542, 12044,  ...,  6176,   202,   102],
        [  101,  3396,  4435,  ...,   179,  7343,   102],
        [  101,  2511,   125,  ...,  5300,   202,   102]])

In [12]:
# Carregar o modelo BERT e configurar o otimizador

# Dispositivo: GPU se disponível, senão CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Carrega o modelo pré-treinado BERT para classificação
model = BertForSequenceClassification.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    num_labels=len(le.classes_)
)

# Atualiza o vocabulário (caso adicionamos [INST])
model.resize_token_embeddings(len(tokenizer))

# Move modelo para a GPU (ou CPU)
model.to(device)

# Define otimizador (AdamW é padrão para transformers)
learning_rate = 5e-5
#optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
optimizer = AdamW(model.parameters(), lr=learning_rate)


Usando dispositivo: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [13]:
# Loop de treinamento (Fine-Tuning)
from tqdm import tqdm

# Parâmetros do treinamento
num_epochs =  5  # (Loss médio da época 5: 0.0832)
model.train()

print("Iniciando treinamento...")

for epoch in range(num_epochs):
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in loop:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        output = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = output.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Loss médio da época {epoch+1}: {total_loss / len(train_loader):.4f}")


Iniciando treinamento...


Epoch 1/5: 100%|██████████| 494/494 [02:45<00:00,  2.98it/s, loss=0.621]


Loss médio da época 1: 0.5992


Epoch 2/5: 100%|██████████| 494/494 [02:45<00:00,  2.99it/s, loss=0.423] 


Loss médio da época 2: 0.3927


...

Loss médio da época 3: 0.2727


Epoch 4/5: 100%|██████████| 494/494 [02:46<00:00,  2.96it/s, loss=0.0235]


Loss médio da época 4: 0.1665


Epoch 5/5: 100%|██████████| 494/494 [02:45<00:00,  2.98it/s, loss=0.132]  

Loss médio da época 5: 0.1114


In [14]:
model.eval()  # coloca o modelo em modo de avaliação

all_preds = []
all_labels = []

with torch.no_grad():  # não precisamos calcular gradientes aqui
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Decodifica os rótulos numéricos de volta para texto
y_pred = le.inverse_transform(all_preds)
y_true = le.inverse_transform(all_labels)

# Avaliação
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

f1 = f1_score(y_true, y_pred, average='weighted')
print(f"[X] F1 Score Final: {f1:.2%}")


Relatório de Classificação:
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.72      0.92      0.81      1252
            Hipotecas / Empréstimos       0.80      0.87      0.83       962
                             Outros       0.76      0.64      0.70       558
       Roubo / Relatório de disputa       0.81      0.77      0.79      1206
         Serviços de conta bancária       0.92      0.73      0.81      1290

                           accuracy                           0.80      5268
                          macro avg       0.80      0.79      0.79      5268
                       weighted avg       0.81      0.80      0.80      5268

[X] F1 Score Final: 79.80%


### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

**Atenção:** **Implemente aqui apenas os scripts que fazem parte do modelo campeão.**


In [15]:
# PIPILINE FINAL: TF-IDF (1-2) + Regressão Logística

# Imports
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Setup NLTK
nltk.download('punkt')
#nltk.download('punkt_tab') # descomentar em caso de problemas 
nltk.download('stopwords')

# dataset
#df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';') 
df = pd.read_csv('tickets_reclamacoes_classificados.csv', sep=';')

# Funções de pré-processamento
stopwords = set(stopwords.words('portuguese'))

def remove_punctuation(text):
    table = str.maketrans({key: " " for key in string.punctuation})
    return text.translate(table)

def norm_tokenize(text):
    text = str(text).lower()
    text = re.sub(r'xx+/xx+/xxxx?', ' ', text)
    text = re.sub(r'\$?\s?\d+(?:,\d+)?', ' ', text)
    text = re.sub(r'(?i)\b[x]{2,}\b', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = remove_punctuation(text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords and len(t) > 2]
    return " ".join(tokens)

# Aplicando pré-processamento
df['texto_processado'] = df['descricao_reclamacao'].apply(norm_tokenize)

# Separação dos dados
X = df['texto_processado']
y = df['categoria']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Vetorização com TF-IDF (1-2)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Modelo campeão: Regressão Logística
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train_vec, y_train)

# Avaliação com F1 Score
y_pred = modelo.predict(X_test_vec)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'´ [ x ] F1 Score do modelo campeão (TF-IDF + Regressão Logística): {f1:.4f}')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


´ [ x ] F1 Score do modelo campeão (TF-IDF + Regressão Logística): 0.9047


### **Conclusão Final**

Parte 1 — Abordagem tradicional de PLN + ML
-Aplicamos técnicas clássicas de pré-processamento, tokenização e vetorização (TF-IDF com N-gramas);
-Utilizamos vários modelos de machine learning para testar e aprender sobre o comportamento;
-Essa abordagem obteve um F1 Score superior a `80%`, tornando-se o melhor desempenho geral

Parte 2 — Abordagem com GenAI (LLM com Transformers)
-Utilizamos o modelo paraphrase-multilingual-MiniLM-L12-v2 para gerar embeddings semânticos com sentence-transformers.
-Também testamos o modelo mais robusto paraphrase-multilingual-mpnet-base-v2, aproveitando aceleração via GPU.
-Foram avaliados diversos classificadores (Logistic Regression, SVC, Random Forest, etc.).
-O melhor F1 Score foi de ~77.98% com LinearSVC + MiniLM, e ~76.96% com mpnet.

Dessa forma, 
apesar do uso de modelos modernos baseados em LLM, o modelo tradicional da Parte 1 obteve desempenho superior em F1 Score. 
Isso mostra que:
 - Em problemas com vocabulário controlado e estrutura previsível, abordagens clássicas ainda podem ser mais eficientes.
 - Modelos baseados em Transformers são promissores, especialmente com fine-tuning específico, mas exigem mais recursos e ajustes.
